# Spotify Listening History Feature Engineering
---
This notebook will:
- Add time-based features
- Compute track and artist play counts
- Session-level features
- Prepare dataset for insight-driven visualizations

In [3]:
#Add
import pandas as pd
import numpy as np

## 1. Load Data
---

In [4]:
#Load cleaned dataset from Notebook 1
df = pd.read_csv("spotify_listening_cleaned.csv")

#View
df.head()

,endTime,msPlayed,trackName,artistName,albumName,secondsPlayed,minutesPlayed
0,2022-07-08 14:28:20+00:00,661,My Fault,Eminem,The Slim Shady LP,0.661,0.011017
1,2022-07-08 14:28:21+00:00,725,Just the Way You Are,Bruno Mars,Doo-Wops & Hooligans,0.725,0.012083
2,2022-07-08 14:28:21+00:00,661,Marsh,Eminem,Music To Be Murdered By - Side B,0.661,0.011017
3,2022-07-08 14:28:22+00:00,682,"Can You Feel the Love Tonight - From ""The Lion...",Joseph Williams,Disney Summer Songs,0.682,0.011367
4,2022-07-08 14:28:23+00:00,703,Angels Don't Fly,Doobie,Faithfully Faded,0.703,0.011717


## 2. Data Sanity Checks
---

In [5]:
#Structure check
df.shape

(31155, 7)

In [6]:
#Structure check
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31155 entries, 0 to 31154
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   endTime        31155 non-null  object 
 1   msPlayed       31155 non-null  int64  
 2   trackName      31155 non-null  object 
 3   artistName     31155 non-null  object 
 4   albumName      31155 non-null  object 
 5   secondsPlayed  31155 non-null  float64
 6   minutesPlayed  31155 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 1.7+ MB


In [7]:
#Time column validation
df['endTime'].min(), df['endTime'].max()

('2022-07-08 14:28:20+00:00', '2025-08-04 19:59:02+00:00')

In [8]:
#Listening duration check
df[['msPlayed', 'secondsPlayed', 'minutesPlayed']].describe()

,msPlayed,secondsPlayed,minutesPlayed
count,3.115500e+04,31155.000000,31155.000000
mean,5.622678e+04,56.226782,0.937113
std,8.738035e+04,87.380346,1.456339
min,2.100000e+01,0.021000,0.000350
25%,2.367500e+03,2.367500,0.039458
50%,8.960000e+03,8.960000,0.149333
75%,7.688500e+04,76.885000,1.281417
max,1.471253e+06,1471.253000,24.520883


### Quick Fix:

- **Problem:** `endTime` is an object, not a datetime
  - **Fix:** Convert to datetime before moving forward

In [9]:
#Quick fix: Convert `endTime` to datetime
df['endTime'] = pd.to_datetime(df['endTime'], utc=True)

In [10]:
#Verify fix
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31155 entries, 0 to 31154
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   endTime        31155 non-null  datetime64[ns, UTC]
 1   msPlayed       31155 non-null  int64              
 2   trackName      31155 non-null  object             
 3   artistName     31155 non-null  object             
 4   albumName      31155 non-null  object             
 5   secondsPlayed  31155 non-null  float64            
 6   minutesPlayed  31155 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(2), int64(1), object(3)
memory usage: 1.7+ MB


## 3. Time-Based Features
---

In [11]:
#Capture temporal listening: additional columns

#Hour of day
df['hour'] = df['endTime'].dt.hour

#Day of week (0: Monday)
df['dayOfWeek'] = df['endTime'].dt.dayofweek

#Day name
df['dayName'] = df['endTime'].dt.day_name()

#Month number
df['month'] = df['endTime'].dt.month

#Year
df['year'] = df['endTime'].dt.year

#Year-Month combination (for trends)
df['yearMonth'] = df['endTime'].dt.to_period('M')

C:\Users\lynnm\AppData\Local\Temp\ipykernel_24564\1920611640.py:19: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['yearMonth'] = df['endTime'].dt.to_period('M')


### Note on `yearMonth` Conversion Warning
The `endTime` column is **timezone-aware (UTC)**, so converting to a Period (month-year) doesn't retain the timezone information. This is safe and can be ignored, given `yearMonth` is only used for grouping and trend analysis (& will not affect it).

## 4. Listening Duartion & Repeat Listening Features
---

In [12]:
#Count the number of plays:

#per track
track_counts = df['trackName'].value_counts().reset_index()
track_counts.columns = ['trackName', 'playCount']

#per artist
artist_counts = df['artistName'].value_counts().reset_index()
artist_counts.columns = ['artistName', 'playCount']

#Add play counts to dataset 
df = df.merge(track_counts, on='trackName', how='left', suffixes=('', '_track'))
df = df.merge(artist_counts, on='artistName', how='left', suffixes=('', '_artist'))

In [13]:
#Define session

#New session starts after 30-min pause
df = df.sort_values('endTime')
df['timeDiff'] = df['endTime'].diff().dt.total_seconds().fillna(0)

#Flag start of a new session (30 min= 1800 sec)
df['newSession'] = df['timeDiff'] > 1800

#Assign them a session ID
df['sessionID'] = df['newSession'].cumsum()

In [14]:
#Verify
df[['endTime','trackName','artistName','secondsPlayed','minutesPlayed','playCount','playCount_artist','timeDiff','sessionID']].head(10)

,endTime,trackName,artistName,secondsPlayed,minutesPlayed,playCount,playCount_artist,timeDiff,sessionID
0,2022-07-08 14:28:20+00:00,My Fault,Eminem,0.661,0.011017,108,2036,0.0,0
1,2022-07-08 14:28:21+00:00,Just the Way You Are,Bruno Mars,0.725,0.012083,44,158,1.0,0
2,2022-07-08 14:28:21+00:00,Marsh,Eminem,0.661,0.011017,42,2036,0.0,0
3,2022-07-08 14:28:22+00:00,"Can You Feel the Love Tonight - From ""The Lion...",Joseph Williams,0.682,0.011367,12,12,1.0,0
4,2022-07-08 14:28:23+00:00,Angels Don't Fly,Doobie,0.703,0.011717,39,1849,1.0,0
5,2022-07-08 14:28:24+00:00,I Got,Burden,0.618,0.010300,9,9,1.0,0
6,2022-07-08 14:28:25+00:00,Heart Attack,Demi Lovato,0.597,0.009950,19,40,1.0,0
7,2022-07-08 14:28:26+00:00,No Idea,Don Toliver,0.576,0.009600,15,23,1.0,0
8,2022-07-08 14:28:26+00:00,Dazed & Confused (feat. Rittz),Chris Webby,0.618,0.010300,9,60,0.0,0
9,2022-07-08 14:28:27+00:00,After Party,Don Toliver,0.703,0.011717,8,23,1.0,0


## 5. Session-Level Features
---

In [15]:
#Calculate total session length in seconds and number of tracks per session
session_stats = df.groupby('sessionID').agg(
    sessionSeconds=('secondsPlayed', 'sum'),
    sessionMinutes=('minutesPlayed', 'sum'),
    tracksInSession=('trackName', 'count')
).reset_index()

#Count repeat listens within each session
repeat_tracks = df.groupby(['sessionID', 'trackName']).size().reset_index(name='trackCount')
repeat_counts = repeat_tracks.groupby('sessionID')['trackCount'].apply(lambda x: (x > 1).sum()).reset_index(name='repeatTracks')

#Merge session-level features back to the main df
df = df.merge(session_stats, on='sessionID', how='left')
df = df.merge(repeat_counts, on='sessionID', how='left')

#Average track length in seconds per session:
df['avgTrackSeconds'] = df['sessionSeconds'] / df['tracksInSession']

In [16]:
#Check column names to confirm
print("\nColumns in dataset:")
print(df.columns)


Columns in dataset:
Index(['endTime', 'msPlayed', 'trackName', 'artistName', 'albumName',
       'secondsPlayed', 'minutesPlayed', 'hour', 'dayOfWeek', 'dayName',
       'month', 'year', 'yearMonth', 'playCount', 'playCount_artist',
       'timeDiff', 'newSession', 'sessionID', 'sessionSeconds',
       'sessionMinutes', 'tracksInSession', 'repeatTracks', 'avgTrackSeconds'],
      dtype='object')


In [17]:
#Save feature-engineered dataset for the next steps
df.to_csv("spotify_listening_featured.csv", index=False)

## Feature Engineering Summary
---
- **Time-Based Features:**
  - Extracted `hour`,`dayOfWeek`, `month`, `year` from `endTime`
- **Track & Artist Play Counts:**
  - Calculated total plays per track and artist
- **Session Features:**
  - Defined sessions with 30-min gap threshold
  - Calculated session length (seconds & minutes)
  - Counted: Tracks per session, repeat tracks (within each session), & average track length (per session)

These features prepare the dataset for insight-driven visualizations.